In [1]:
from __future__ import print_function

import tensorrt as trt
import os
import sys
import onnx

import _trt_path

import common

In [2]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

In [3]:
def get_engine(onnx_file_path, engine_file_path=""):
    def build_engine():
        with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
            builder.max_workspace_size = 1<<30
            builder.max_batch_size = 1

            if not os.path.exists(onnx_file_path):
                print('ONNX file {} not found'.format(onnx_file_path))
                exit(0)
            print('Loading ONNX file from path {}...'.format(onnx_file_path))
            with open(onnx_file_path, 'rb') as model:
                print('Beginning ONNX file parsing')
                parser.parse(model.read())
            print('Completed parsing of ONNX file')
            print('Building an engine from file {}; this may take a while...'.format(onnx_file_path))

            engine = builder.build_cuda_engine(network)
            print("Completed creating Engine")
            with open(engine_file_path, "wb") as f:
                f.write(engine.serialize())
            return engine
        
    if os.path.exists(engine_file_path):
        print("Reading engine from file {}".format(engine_file_path))
        with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
            return runtime.deserialize_cuda_engine(f.read())
    else:
        return build_engine()

In [ ]:
get_engine("centernet.onnx", "centernet.trt")

In [1]:
import onnx
import onnx_tensorrt.backend as backend
import numpy as np

model = onnx.load("centernet.onnx")
engine = backend.prepare(model, device='CUDA:1')
input_data = np.random.random(size=(16, 3, 512, 512)).astype(np.float32)
output_data = engine.run(input_data)[0]
print(output_data)
print(output_data.shape)

ModuleNotFoundError: No module named 'onnx_tensorrt'